In [1]:
import numpy as np
import imutils
import pickle
import cv2
import os

In [2]:
# Prepare model
modelFolder = "./face_detection_model"
protoFile = os.path.sep.join([modelFolder, "deploy.prototxt"])
modelFile = os.path.sep.join([modelFolder, "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoFile, modelFile)

# Prepare embedded
embeddedFile = "./openface_nn4.small2.v1.t7"
embedder = cv2.dnn.readNetFromTorch(embeddedFile)

In [3]:
# Prepare recognizer
recognizerFile = "output/recognizer.pickle"
labelEncoderFile = "output/label_encoder.pickle"
recognizer = pickle.loads(open(recognizerFile, "rb").read())
labelEncoder = pickle.loads(open(labelEncoderFile, "rb").read())

In [4]:
# Prepare input image
inputImageFile = "images/000.jpg"
orgImage = cv2.imread(inputImageFile)
(orgHeight, orgWidth) = orgImage.shape[:2]
print("[original] width={0} height={1}".format(orgWidth, orgHeight))

# Show original
# cv2.imshow("Original", orgImage)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[original] width=3456 height=4608


In [5]:
# Resize input image (maintain the aspect ratio)
image = imutils.resize(orgImage, width=600)
(height, width) = image.shape[:2]

# Resize image to 300x300 pixels to blob object and normalizing
outWidth, outHeight = 300, 300
resizedImage = cv2.resize(image, (outWidth, outHeight))
scale = 1.0
mean = (104.0, 177.0, 123.0)

# Create an image blob
imageBlob = cv2.dnn.blobFromImage(resizedImage, scale, (outWidth, outHeight), mean)

In [6]:
# Set minimum confidence
minimumConfidence = 0.5

# Prepare for drawing
borderColor = (255, 0, 255)
textColor = (0, 0, 255)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.45
thickness = 2
# Prepare face parameters
faceMinWidth, faceMinHeight = 20, 20
faceOutputWidth, faceOutputHeight = 96, 96
faceScale = 1.0 / 255
faceMean = (0, 0, 0)

In [7]:
# copy image
output = image.copy()
# Pass the blob through network
detector.setInput(imageBlob)
detections = detector.forward()
# loop over the detections
numDetections = detections.shape[2]
print("numDetections={0}".format(numDetections))

# Face detection
for i in range(0, numDetections):
    # extract the confidence associated with the prediction
    confidence = detections[0, 0, i, 2]
    if confidence > minimumConfidence:
        # compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
        (startX, startY, endX, endY) = box.astype("int")
        # draw rectangle with the associated propbability
        cv2.rectangle(output, (startX, startY), (endX, endY), borderColor, thickness)

        text = "{:.2f}%".format(confidence * 100)
        textY = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(output, text, (startX, textY), font, fontScale, textColor, thickness)

# show the output image
cv2.imshow("Output", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

numDetections=200


In [8]:
# copy image
output = image.copy()
# Pass the blob through network
detector.setInput(imageBlob)
detections = detector.forward()
# loop over the detections
numDetections = detections.shape[2]
print("numDetections={0}".format(numDetections))

# Face detection
for i in range(0, numDetections):
    # extract the confidence associated with the prediction
    confidence = detections[0, 0, i, 2]
    if confidence > minimumConfidence:
        # compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
        (startX, startY, endX, endY) = box.astype("int")

        # extract the face ROI
        face = image[startY:endY, startX:endX]
        (faceHeight, faceWidth) = face.shape[:2]
        # ensure the face width and height are sufficiently large
        if faceWidth < faceMinWidth or faceHeight < faceMinHeight:
            continue
        
        # Create a blob from the face ROI
        faceBlob = cv2.dnn.blobFromImage(face, faceScale, (faceOutputWidth, faceOutputHeight), faceMean, swapRB=True, crop=False)

        # Who is this?
        embedder.setInput(faceBlob)
        vector = embedder.forward()

        # Face classification
        preds = recognizer.predict_proba(vector)[0]
        k = np.argmax(preds)
        probability = preds[k]
        label = labelEncoder.classes_[k]

        # draw rectangle with the associated propbability
        cv2.rectangle(output, (startX, startY), (endX, endY), borderColor, thickness)

        text = "{0} - {1:.2f}%".format(label, probability * 100)
        textY = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(output, text, (startX, textY), font, fontScale, textColor, thickness)

# show the output image
cv2.imshow("Output", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

numDetections=200
